# Hyperparameter Tuning using HyperDrive

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
import joblib

import collections
# Due to the not working widgets library at Python 3.10+
if not hasattr(collections, 'Mapping'):
    import collections.abc
    collections.Mapping = collections.abc.Mapping
    collections.Sequence = collections.abc.Sequence
    collections.Iterable = collections.abc.Iterable
    collections.Callable = collections.abc.Callable

from azureml.widgets import RunDetails

## Dataset

Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

experiment_name = 'Hyperdrive-HFP-Exp'

experiment=Experiment(ws, experiment_name)
experiment

quick-starts-ws-294022
aml-quickstarts-294022
westeurope
6971f5ac-8af1-446e-8034-05acea24681f


Name,Workspace,Report Page,Docs Page
Hyperdrive-HFP-Exp,quick-starts-ws-294022,Link to Azure Machine Learning studio,Link to Documentation


In [3]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "heart_failure_clinical_records"
description_text = ""

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://github.com/onuroezcelik/nd00333_Capstone_Project_Machine_Learning_Engineer/blob/master/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

print("The registered dataset is found: ",found)
df = dataset.to_pandas_dataframe()
df.describe()

The registered dataset is found:  True
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [4]:
dataset.take(5).to_pandas_dataframe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## Hyperdrive Configuration

Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
# Choose a name for your CPU cluster
amlcompute_cluster_name = "automlcluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster.')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [6]:
# Specify a Policy
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.2)

# Specify parameter sampler
param_sampling = RandomParameterSampling(
    {
        '--C' : choice(0.1,1),
        '--max_iter': choice(10,20,50)
    }
)

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Estimator and hyperdrive config
estimator = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=amlcompute_cluster_name,
    environment=sklearn_env    
    )

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=early_termination_policy,
                                     run_config=estimator,
                                     max_total_runs=6,
                                     max_concurrent_runs=4)

In [7]:
#Submit experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

2026-01-19 11:37:03.216681: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768822623.238935   28716 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768822623.245623   28716 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768822623.262529   28716 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768822623.262559   28716 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768822623.262561   28716 computation_placer.cc:177] computation placer alr

A Jupyter Widget

RunId: HD_6f032a47-7da5-42f1-aa15-60d6ad2b04f0
Web View: https://ml.azure.com/runs/HD_6f032a47-7da5-42f1-aa15-60d6ad2b04f0?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-294022/workspaces/quick-starts-ws-294022&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2026-01-19T11:36:41.3464703Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2026-01-19T11:36:41.8169431Z][SCHEDULER][INFO]Scheduling job, id='HD_6f032a47-7da5-42f1-aa15-60d6ad2b04f0_0' 
[2026-01-19T11:36:41.9318905Z][SCHEDULER][INFO]Scheduling job, id='HD_6f032a47-7da5-42f1-aa15-60d6ad2b04f0_1' 
[2026-01-19T11:36:41.9311243Z][SCHEDULER][INFO]Scheduling job, id='HD_6f032a47-7da5-42f1-aa15-60d6ad2b04f0_2' 
[2026-01-19T11:36:42.0502061Z][SCHEDULER][INFO]Scheduling job, id='HD_6f032a47-7da5-42f1-aa15-60d6ad2b04f0_3' 
[2026-01-19T11:36:42.5296048Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_6f032a47-7da5-42f1-aa15-60d6ad2b04f0_0' 
[2026-01-19T11:3

{'runId': 'HD_6f032a47-7da5-42f1-aa15-60d6ad2b04f0',
 'target': 'automlcluster',
 'status': 'Completed',
 'startTimeUtc': '2026-01-19T11:36:39.604168Z',
 'endTimeUtc': '2026-01-19T11:52:19.33467Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '6d83ca46-8656-4b93-b49d-9f8af4350b12',
  'user_agent': 'python/3.10.11 (Linux-6.8.0-1030-azure-x86_64-with-glibc2.35) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.60.0',
  'best_child_run_id': 'HD_6f032a47-7da5-42f1-aa15-60d6ad2b04f0_3',
  'score': '0.7666666666666667',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_6f032a47-7da5-42f1-aa15-60d6ad2b04f0_3'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-t

## Best Model

In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [9]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics() 
best_run_file_names = best_run.get_file_names()

print('Best Run Id: ', best_run.id) 
print('Best Run Metrics: ', best_run_metrics) 
print('Best Run Accuracy:', best_run_metrics['Accuracy']) 
print('Best Run File Names:', best_run_file_names) 

Best Run Id:  HD_6f032a47-7da5-42f1-aa15-60d6ad2b04f0_3
Best Run Metrics:  {'Regularization Strength:': 0.1, 'Max iterations:': 50, 'Accuracy': 0.7666666666666667}
Best Run Accuracy: 0.7666666666666667
Best Run File Names: ['logs/azureml/dataprep/0/rslex.log.2026-01-19-11', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [10]:
# Save the best model
hd_model = best_run.register_model(model_name='best_hd_model', model_path='outputs/')